In [1]:
import re
from typing import Set, List
from dataclasses import dataclass, field


In [2]:
sample = """seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""



@dataclass
class MapRange:
    dest_start:int
    source_start:int
    length:int

    def applies_to(self, x):
        return x >= self.source_start and x < self.source_start + self.length

    def apply(self,x):
        res = x-self.source_start + self.dest_start
        print(f'Applying {self} to {x}. Result: {res}')

        return res

    def rev_applies_to(self,x):
        return self.dest_start <= x < self.dest_start+self.length

    def revert(self, x):
        return x - self.dest_start + self.source_start


assert not MapRange(3,4,2).applies_to(3)
assert MapRange(3,4,2).applies_to(4)
assert MapRange(3,4,2).applies_to(5)
assert not MapRange(3,4,2).applies_to(6)

@dataclass
class Mapping:
    source:str
    target:str
    ranges: list[MapRange] = field(default_factory=list)


mapping_section_pattern = r"(\w+)-to-(\w+) map:"
range_pattern = r"^\s*(\d+)\s+(\d+)\s+(\d+)"
seeds_pattern = r"seeds: (?:(\d+)\s+)+"

def parse_input(puzzle_input):
    seeds = []
    mappings = []

    for line in puzzle_input.split('\n'):
        if m :=  re.search(seeds_pattern,line):
            seeds = list(map(lambda x: int(x.strip()), line.split(':')[-1].strip().split(' ')))
        if m := re.search(mapping_section_pattern, line):
            mappings.append(Mapping(*m.groups()))
        if m := re.search(range_pattern, line):
            mappings[-1].ranges.append(MapRange(*map(int, m.groups())))

    return seeds, mappings


def get_location(seed_number, mappings):
    current_value = seed_number
    print(len(mappings))
    for m in mappings:
        print(m.source, m.target)
        applicable_ranges = list(filter(lambda x: x.applies_to(current_value), m.ranges))
        #print(applicable_ranges)
        if len(applicable_ranges)==1:
            current_value = applicable_ranges[0].apply(current_value)
    return current_value
        
    
def solve(puzzle):
    seeds, mappings = parse_input(puzzle)

    locations = list(map(lambda x : get_location(x, mappings), seeds))

    print(locations)
    return min(locations)                      

In [3]:
solve(sample)

7
seed soil
Applying MapRange(dest_start=52, source_start=50, length=48) to 79. Result: 81
soil fertilizer
fertilizer water
water light
Applying MapRange(dest_start=18, source_start=25, length=70) to 81. Result: 74
light temperature
Applying MapRange(dest_start=68, source_start=64, length=13) to 74. Result: 78
temperature humidity
humidity location
Applying MapRange(dest_start=60, source_start=56, length=37) to 78. Result: 82
7
seed soil
soil fertilizer
Applying MapRange(dest_start=39, source_start=0, length=15) to 14. Result: 53
fertilizer water
Applying MapRange(dest_start=49, source_start=53, length=8) to 53. Result: 49
water light
Applying MapRange(dest_start=18, source_start=25, length=70) to 49. Result: 42
light temperature
temperature humidity
Applying MapRange(dest_start=1, source_start=0, length=69) to 42. Result: 43
humidity location
7
seed soil
Applying MapRange(dest_start=52, source_start=50, length=48) to 55. Result: 57
soil fertilizer
fertilizer water
Applying MapRange(de

35

In [4]:
with open('input_5.txt','r') as infile:
    puzzle = infile.read()
solve(puzzle)

7
seed soil
Applying MapRange(dest_start=3840286095, source_start=2849853212, length=361877006) to 3139431799. Result: 4129864682
soil fertilizer
Applying MapRange(dest_start=3581406499, source_start=4034715214, length=260252082) to 4129864682. Result: 3676555967
fertilizer water
Applying MapRange(dest_start=3996287530, source_start=3557602002, length=204061128) to 3676555967. Result: 4115241495
water light
Applying MapRange(dest_start=3487999223, source_start=4080968704, length=155844998) to 4115241495. Result: 3522272014
light temperature
Applying MapRange(dest_start=3495346659, source_start=3466620869, length=376755291) to 3522272014. Result: 3550997804
temperature humidity
Applying MapRange(dest_start=3966168141, source_start=3406025946, length=214996780) to 3550997804. Result: 4111139999
humidity location
7
seed soil
Applying MapRange(dest_start=1335949488, source_start=0, length=69797365) to 50198205. Result: 1386147693
soil fertilizer
Applying MapRange(dest_start=890092371, sour

462648396

In [48]:
import itertools
from tqdm.notebook import tqdm

def solve2(puzzle):
    seeds, mappings = parse_input(puzzle)

    def in_initial_seeds(x,seed_list):
        for start, len in  list(itertools.islice(itertools.pairwise(seed_list),0, None, 2)):
            #print(x, start, len)
            if start > x: return False
            if start+len <= x: return False
            return True

    
    assert in_initial_seeds(1, [1, 3])
    assert not in_initial_seeds(0, [1, 3])
    assert not in_initial_seeds(4, [1, 3])

    def reverse(x,mappings):
        current_value = x
        for m in mappings[-1::-1]:
            applicable_ranges = list(filter(lambda x: x.rev_applies_to(current_value), m.ranges))
            #print(applicable_ranges)
            if len(applicable_ranges) == 1:
                current_value = applicable_ranges[0].revert(current_value)
        return current_value

    i = 1
    pb = tqdm()
    while True:
        seed = reverse(i,mappings)
        if in_initial_seeds(seed,seeds):
            print(f'Location {i} maps to seed {seed}')
            return seed
        i+=1
        if i% 5_000 == 0:
            pb.update(5000)
    pb.close()

In [49]:
%time solve2(sample)

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [43]:
%time solve2(puzzle)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 30 µs



0it [00:00, ?it/s]
5000it [00:00, 12974.76it/s]
10000it [00:00, 14737.01it/s]
15000it [00:00, 15792.09it/s]
20000it [00:01, 16210.27it/s]
25000it [00:01, 16207.82it/s]
30000it [00:01, 16608.01it/s]
35000it [00:02, 16593.48it/s]
40000it [00:02, 16456.41it/s]
45000it [00:02, 16085.17it/s]
50000it [00:03, 16086.94it/s]
55000it [00:03, 14736.66it/s]
60000it [00:03, 14985.66it/s]
65000it [00:04, 15318.77it/s]
70000it [00:04, 14879.25it/s]
75000it [00:04, 15480.90it/s]
80000it [00:05, 15656.30it/s]
85000it [00:05, 15787.21it/s]
90000it [00:05, 15883.66it/s]
95000it [00:06, 16368.12it/s]
100000it [00:06, 16350.36it/s]
105000it [00:06, 16484.83it/s]
110000it [00:06, 16597.89it/s]
115000it [00:07, 16767.43it/s]
120000it [00:07, 16622.52it/s]
125000it [00:07, 16633.81it/s]
130000it [00:08, 17028.93it/s]
135000it [00:08, 15513.64it/s]
140000it [00:08, 14967.94it/s]
145000it [00:09, 15495.75it/s]
150000it [00:09, 14363.87it/s]
155000it [00:09, 14532.77it/s]
160000it [00:10, 15186.63it/s]
165000it

KeyboardInterrupt: 

In [47]:
import sys
import re
from collections import defaultdict

D = puzzle.strip()
L = D.split('\n')

parts = D.split('\n\n')
seed, *others = parts
seed = [int(x) for x in seed.split(':')[1].split()]

class Function:
  def __init__(self, S):
    lines = S.split('\n')[1:] # throw away name
    # dst src sz
    self.tuples: list[tuple[int,int,int]] = [[int(x) for x in line.split()] for line in lines]
    #print(self.tuples)
  def apply_one(self, x: int) -> int:
    for (dst, src, sz) in self.tuples:
      if src<=x<src+sz:
        return x+dst-src
    return x

  # list of [start, end) ranges
  def apply_range(self, R):
    A = []
    for (dest, src, sz) in self.tuples:
      src_end = src+sz
      NR = []
      while R:
        # [st                                     ed)
        #          [src       src_end]
        # [BEFORE ][INTER            ][AFTER        )
        (st,ed) = R.pop()
        # (src,sz) might cut (st,ed)
        before = (st,min(ed,src))
        inter = (max(st, src), min(src_end, ed))
        after = (max(src_end, st), ed)
        if before[1]>before[0]:
          NR.append(before)
        if inter[1]>inter[0]:
          A.append((inter[0]-src+dest, inter[1]-src+dest))
        if after[1]>after[0]:
          NR.append(after)
      R = NR
    return A+R

Fs = [Function(s) for s in others]

def f(R, o):
  A = []
  for line in o:
    dest,src,sz = [int(x) for x in line.split()]
    src_end = src+sz

P1 = []
for x in seed:
  for f in Fs:
    x = f.apply_one(x)
  P1.append(x)
print(min(P1))

P2 = []
pairs = list(zip(seed[::2], seed[1::2]))
for st, sz in pairs:
  # inclusive on the left, exclusive on the right
  # e.g. [1,3) = [1,2]
  # length of [a,b) = b-a
  # [a,b) + [b,c) = [a,c)
  R = [(st, st+sz)]
  for f in Fs:
    R = f.apply_range(R)
  #print(len(R))
  P2.append(min(R)[0])
print(min(P2))

#Taken from:
#https://github.com/jonathanpaulson/AdventOfCode/blob/master/2023/5.py

462648396
2520479
